In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install cv2_imshow
!pip install paddlepaddle
# !python3 -m pip install paddlepaddle-gpu
!pip install paddleocr
!pip install ultralytics

In [ ]:
from google.colab.patches import cv2_imshow
from paddleocr import PaddleOCR
import os
import csv
import requests
import cv2

import requests
from PIL import Image
import io
from ultralytics import YOLO

from datetime import datetime
import pandas as pd
import numpy as np
import re
from functools import reduce

In [ ]:
!pip freeze > requirements.txt
!pip list --format=freeze > requirements.txt

In [ ]:
# Load a model
best_model_path = '/content/drive/MyDrive/iFarmer/PROJECT-NID-OCR/runs/detect/train5/weights/best.pt'
model = YOLO(best_model_path) # Load the pretrained model

In [ ]:
def nid_ocr(url):
    ocr = PaddleOCR(use_angle_cls=True, lang='en')
    image_url = url
    nid_conf = ""
    dob_conf = ""

    nid = ""
    dob = ""
    try:
        # Download the image from the URL
        response = requests.get(image_url)
        image = Image.open(io.BytesIO(response.content))
        results = model(image, save=False, save_crop=False)

        image_np = np.array(image)
        image = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

        # print(image)

        k = 0
        l = 0

        # Check if any objects are detected in the image
        for result in results:
            if image is not None:
              # Image name extraction
                # image_name = os.path.splitext(os.path.basename(result.path))[0]
                boxes = result.boxes.cpu().numpy()

                if len(boxes) >= 2:
                    for box in boxes:
                        region = box.xyxy[0].astype(int)

                        cropped_image = image[region[1]:region[3], region[0]:region[2]]

                        # Generate a unique filename for the cropped image
                        class_val = box.cls
                        # print("1")

                        # Generate a unique filename for the cropped image
                        # filename = f"{row['nid']}_class_{class_val[0]}.jpg"
                        # output_path = os.path.join(output_folder, filename)

                        try:

                            # Perform OCR on the cropped image
                            ocr_results = ocr.ocr(cropped_image, cls=True)

                            # Process OCR results
                            if (class_val == 0) and (k < 1):
                                nid_conf = str(ocr_results[0][0][1][1])
                                nid = str(ocr_results[0][0][1][0])
                                nid = re.sub(r'[^A-Za-z0-9]+', '', nid)
                                k+=1
                                # print(nid)

                            elif (class_val) == 1 and (l < 1):
                                dob_conf = str(ocr_results[0][0][1][1])
                                dob = str(ocr_results[0][0][1][0])
                                dob = re.sub(r'[^A-Za-z0-9]+', '', dob)
                                l+=1


                        except Exception as e:
                            print("Can Not Process")

                else:
                  print("Can Not Process")
            else:
              print("Can Not Process")

    except Exception as e:
        print("Can Not Process")

    return nid, dob, nid_conf, dob_conf

In [ ]:
def convert_date(date_str):
    # Define the mapping of month abbreviations to month numbers
    month_map = {
        'Jan': '01',
        'Feb': '02',
        'Mar': '03',
        'Apr': '04',
        'May': '05',
        'Jun': '06',
        'Jul': '07',
        'Aug': '08',
        'Sep': '09',
        'Oct': '10',
        'Nov': '11',
        'Dec': '12'
    }

    # Convert the date string to datetime object
    date_obj = datetime.strptime(date_str, "%d%b%Y")

    # Extract the year, month, and day components
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day

    # Format the date components to have leading zeros if necessary
    year_str = str(year)
    month_str = month_map[date_obj.strftime("%b")]
    day_str = str(day).zfill(2)

    # Create the formatted date string in "YYYY-MM-DD" format
    formatted_date = f"{year_str}-{month_str}-{day_str}"

    return formatted_date

In [ ]:
def mon_start_end(month):
    mon = month
    if mon.startswith("Ja"):    return '-01-'
    elif mon.startswith(("Fe", "FE" ,"fe")):    return '-02-'
    elif mon.startswith("Mar"):    return '-03-'
    elif mon.startswith(("Ap", "ap", "AP")):    return '-04-'
    elif mon.startswith(("MAY", "May", "MaY")):   return '-05-'
    elif mon.startswith(("Jun", "JUN", "JUn")):   return '-06-'
    elif mon.startswith(("Jul", "JUl")):   return '-07-'
    elif mon.startswith(("Au", "AUg", "AuG")):    return '-08-'
    elif mon.startswith(("Se", "se", "5e", "Sep")):   return '-09-'
    elif mon.startswith(("Oc","Oct")):    return '-10-'
    elif mon.startswith(("No", "NO", "Nov")):   return '-11-'
    elif mon.startswith(("De", "Dec")):   return '-12-'

    elif mon.endswith("an"):   return '-01-'
    elif mon.endswith("eb"):   return '-02-'
    elif mon.endswith("ar"):   return '-03-'
    elif mon.endswith("pr"):   return '-04-'
    elif mon.endswith("ay"):   return '-05-'
    elif mon.endswith("un"):   return '-06-'
    elif mon.endswith("ul"):   return '-07-'
    elif mon.endswith(("ug","0g","Ug")):   return '-08-'
    elif mon.endswith("ep"):   return '-09-'
    elif mon.endswith("ct"):   return '-10-'
    elif mon.endswith("ov"):   return '-11-'
    elif mon.endswith("ec"):   return '-12-'

    elif mon.startswith(("Ja", "J")) and mon.endswith("an"):    return '-01-'
    elif mon.startswith(("F", "Fe")) and mon.endswith(("eb","b")):    return '-02-'
    elif mon.startswith(("M", "Ma")) and mon.endswith(("ar","r")):    return '-03-'
    elif mon.startswith(("Ap", "A")) and mon.endswith(("r","pr")):    return '-04-'
    elif mon.startswith(("Ma", "M")) and mon.endswith(("ay","y")):    return '-05-'
    elif mon.startswith(("Ju", "J")) and mon.endswith("un"):    return '-06-'
    elif mon.startswith(("JU", "J")) and mon.endswith(("ul","l")):    return '-07-'
    elif mon.startswith(("Au", "A", "AU")) and mon.endswith(("ug","g","9")):    return '-08-'
    elif mon.startswith(("Se", "S", "8")) and mon.endswith(("ep","p","0")):    return '-09-'
    elif mon.startswith(("Oc", "O")) and mon.endswith(("ct","t")):    return '-10-'
    elif mon.startswith(("No", "N")) and mon.endswith(("ov","v")):    return '-11-'
    elif mon.startswith(("De", "D")) and mon.endswith(("ec,c")):    return '-12-'

    return None

In [ ]:
def convert_mispelled_string_to_month(mispelled_string):
    # Define a mapping of correct month names to their abbreviations
    month_mapping = {
        '-01-': ['Jan', 'jan', 'an', 'Ja', '3an', 'Jar', 'lan', 'Jsn','oan','Jen','dan','Jah'],
        '-02-': ['Feb', 'FoD','F6b','feb', 'F', 'Fb', 'Fe', 'eb', 'b', 'Fob', 'Fo', 'Feh', '5ob', 'Fab', 'Fa','Fsh','Fcb','Fo6','Fet','FeD','Fe6','Teb'],
        '-03-': ['Mar', 'mar', 'Mer', 'Mr', 'ar', 'Man', 'Mor','Var','Mat'],
        '-04-': ['Apr', 'apr', 'Ap','Ar','pr','Aor','ADr','Abr','AD','ApE','Apf','Apt','AOr', 'or', '4or', 'nr'],
        '-05-': ['May', 'may', 'M5Y', 'Mey', 'My', 'ay', 'Mav', 'Vay', 'y', 'Moy', 'MaV', 'M99', 'M9v', 'av'],
        '-06-': ['Jun', 'jun','Jum', 'un', 'vn', 'Jur','June','ur'],
        '-07-': ['Jul', 'jul', 'Jol', 'Jl', 'JL', 'ul', 'Jut','JuI','Jui','3ul', '1ul', 'uT', 'JuT', 'J01', 'U1', 'u1'],
        '-08-': ['Aug', 'Au', 'Auc','A0g', 'ug', 'Ag', 'Auq', 'AU', 'Aua', 'Auo', 'Aup', 'AU0','Aud','AUD','Aog','Ang','A00', '40S'],
        '-09-': ['Sep', '8e','ep', 'Sp', 'Seo', 'Sop', '3ep', 'So', 'Sen', 'Sco', 'Sap', 'Se0','Scp','S0p', '5eP', '5op', 'en', '0p', 'S60'],
        '-10-': ['Oct', 'oct', '0c', 'Oc','ct','Ot','t','6ct','Qct','Ocf','0ot','Oo','Ocl','0ct','Oot','Odt','Cct','Oet','5ct','00t','Act'],
        '-11-': ['Nov', 'NOV' 'No', 'ov', 'N', 'Nv', 'oV', 'Noy','NoV','Jov','Nwv', 'Ncv', 'N6v'],
        '-12-': ['Dec', '0ec','Jec', 'De', 'Dc', 'ec', 'Doc', 'Dac', 'Oe', 'Deo', 'DoC', 'DC','eC','D6','D6c','6c']
    }

    mon = mon_start_end(mispelled_string)
    if mon != None:
        return mon
    for month, abbreviations in month_mapping.items():
        # Check if the mispelled_string is in the list of abbreviations
        if mispelled_string in abbreviations:
            return month
    for month, abbreviations in month_mapping.items():
        # Check if the mispelled_string is in the list of abbreviations
        for i in range(len(abbreviations)):
            if abbreviations[i] in mispelled_string:
                return month  # Return the correct month name

    return None

In [ ]:
def mon_corr(month):
    try:
        key = month
        if len(key) >=1:
            month = convert_mispelled_string_to_month(key)
            if month == None:
                month = "NA"
                return month
            else:
                return month
        else:
            month = "NA"
            return month
    except:
        month = "NA"
        return month

In [ ]:
mon_corr('111')

In [ ]:
def day_corr(day):
    string = day
    replacements = [
        ("T", "1"), ("O", "0"), ("o", "0"), ("Q", "0"), ("C", "0"), ("B", "8"), ("Z", "2"), ("U", "0"), ("n", "0"),
        ("l", "1"), ("P", "2"), ("I", "1"), ("s", "8"), ("t", "1")
    ]

    result = reduce(lambda x, y: x.replace(y[0], y[1]), replacements, string)
    return result

In [ ]:
def year_corr(string):

    try:
        zero = ["0","C","c","O","o","D"]
        new_yr = ""
        temp =""
        my_string = string
        if len(my_string) != 4:
            new_yr = "NA"
            return new_yr
        a,b,c,d = my_string
        if (my_string[1] == "1" or my_string[2] == "1"):
            new_yr = "NA"
            return new_yr

        elif (a == "1" or b == "9") and (c in zero):
            new_yr = "NA"
            return new_yr

        elif not (my_string[1] == "1" or my_string[2] == "1"):
            # if not my_string[:2].isdigit():
            if my_string[-2:].isdigit():
                if (my_string[-2:] > "23"):
                    new_yr = "19" + my_string[-2:]
                    return new_yr
                elif my_string[-2:] < "10" and my_string[1]!="2"::
                    new_yr = "20" + my_string[-2:]
                    return new_yr
                else:
                    new_yr = "NA"
                    return new_yr
            else:
                if c == 't':    c = '6'
                if c == 'g':    c = '9'
                if d == 'a':    d = '8'
                if d == 'g':    d = '9'
                if a == 'j' or a == 'i' or a == 'I':    a = '1'
                if a == '1':    b = '9'
                if a == '2':    b = '0'
                if b == '9':    a = '1'
                if b == '0' and (c+d) < '10' :    a = '2'
                if b == '0' and c == 'D':   c = '0'
                if c == 'Q' and b != '0':    c = '9'
                if c == 'Q' and b == '0':    c = '0'
                if c == 'C' and b == '0':    c = '0'
                if c == 'c' and b == '0':    c = '0'
                if c == 'D':    c == '0'
                if c == 'B' or c == 'x':    c == '8'
                if d == 'D':    d == '0'
                if d == 'B':    d == '8'
                if d == 'C' or d == 'c':    d = '0'
                if d == 'i' or d == 'I' or d == 'T':    d = '1'

                temp = c+d
                if temp.isdigit():
                    if (temp > "23"):
                        new_yr = "19" + temp
                        return new_yr
                    elif temp < "10":
                        new_yr = "20" + temp
                        return new_yr
                else:
                    new_yr = "NA"
                    return new_yr
    except:
        new_yr = "NA"
        return new_yr

In [ ]:
def year_cor(year =''):
    try:
        string = year
        # temp = ""
        if (string.isdigit()) and (string > "1920") and (string < "2008"):
            mod_year = string
        else:
            year = year_corr(string)

            mod_year = year
    except:
        mod_year = "NA"

    return mod_year

In [ ]:
def eight_dig(string):
    if string[1] == "O":    df["DOB_Corr"][i] = "NA"
    elif

In [ ]:
def dob_cor(dob):
    try:
        string = dob

        day = string[:2]
        month = string[2:-4]
        year = string[-4:]

        month = mon_corr(month)
        day = day_corr(day)
        year = year_cor(year)

        if (len(string) <=8) and (string[1] == "O"):
            dob_corr = "NA"

        elif day.isdigit() and year.isdigit() and (month != "ND" and month != "NA"):
            a,b = str(day)
            if (a+b) > "31":
                if a == "3":
                    day = "0"+b
                    dob_corr = year + month + day
                elif a == "9":
                    day = "0"+b
                    dob_corr = year + month + day
                elif a == "7":
                    day = "2"+b
                    dob_corr = year + month + day
            elif (a+b) <= "00":
                dob_corr = "NA"
            else:
                dob_corr = year + month + day
        else:
            dob_corr = "NA"
    except:
        dob_corr = "NA"
    return dob_corr

In [ ]:
def nid_cor(nid):
    string = nid
    string = ''.join(map(lambda x: x.replace("U", "0").replace("O", "0").replace("o", "0").replace("/", "7").replace("G", "6"), string))
    if string.isdigit():
        return string
    return 'NA'

In [ ]:
def main(url):
    nid, dob, nid_conf, dob_conf = nid_ocr(url)
    nid_corr = nid_cor(nid)
    try:
        formatted_date = convert_date(dob) #"NA"
    except:
        formatted_date = dob_cor(dob)

    if nid_corr == 'NA' or formatted_date == 'NA':
        print("Give a clear image")

    else:
        return nid_corr, nid_conf, formatted_date, dob_conf


In [ ]:
# main("give image link here")